In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
spark = SparkSession.builder.appName("Operations").getOrCreate()
df = spark.read.csv('/FileStore/tables/wc2018_players.csv',inferSchema=True,header=True)

In [0]:
df.show()

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
|Argentina|  4|  DF|  ANSALDI Cristian|20.09.1986|   ANSALDI|     Torino FC (ITA)|   181|    73|
|Argentina|  5|  MF|      BIGLIA Lucas|30.01.1986|    BIGLIA|      AC Milan (ITA)|   175|    73|
|Argentina|  7|  MF|       BAN

1.    Shows the names and height by adding 1 to the height column.
2.    shows the player name and simultaneously checks whether or not they have height >170
3.    Show FIFA Popular Name and 0 or 1 depending on Height>170
4.    name of  shortest player 
5.    who is tallest of all. First we find the value of maximum height and then get the details of that player
6.    average height of the players in Argentina team.

In [0]:
df.columns

Out[8]: ['Team',
 '#',
 'Pos.',
 'FIFA Popular Name',
 'Birth Date',
 'Shirt Name',
 'Club',
 'Height',
 'Weight']

1 Shows the names and height by adding 1 to the height column.

In [0]:
df.select('FIFA Popular Name',df['Height']+1).show()

+------------------+------------+
| FIFA Popular Name|(Height + 1)|
+------------------+------------+
|TAGLIAFICO Nicolas|         170|
|    PAVON Cristian|         170|
|    LANZINI Manuel|         168|
|    SALVIO Eduardo|         168|
|      MESSI Lionel|         171|
|  ANSALDI Cristian|         182|
|      BIGLIA Lucas|         176|
|       BANEGA Ever|         176|
| MASCHERANO Javier|         175|
|      DYBALA Paulo|         178|
|     AGUERO Sergio|         173|
|   HIGUAIN Gonzalo|         185|
|    DI MARIA Angel|         179|
|  LO CELSO Giovani|         178|
|  MEZA Maximiliano|         181|
|      ACUNA Marcos|         173|
|CABALLERO Wilfredo|         187|
|   MERCADO Gabriel|         182|
|  OTAMENDI Nicolas|         182|
|       ROJO Marcos|         190|
+------------------+------------+
only showing top 20 rows



2 shows the player name and simultaneously checks whether or not they have height >170

In [0]:
df.select('FIFA Popular Name').filter(df['Height'] > 170).show()

+------------------+
| FIFA Popular Name|
+------------------+
|  ANSALDI Cristian|
|      BIGLIA Lucas|
|       BANEGA Ever|
| MASCHERANO Javier|
|      DYBALA Paulo|
|     AGUERO Sergio|
|   HIGUAIN Gonzalo|
|    DI MARIA Angel|
|  LO CELSO Giovani|
|  MEZA Maximiliano|
|      ACUNA Marcos|
|CABALLERO Wilfredo|
|   MERCADO Gabriel|
|  OTAMENDI Nicolas|
|       ROJO Marcos|
|    FAZIO Federico|
|     ARMANI Franco|
|     GUZMAN Nahuel|
|      KRUSE Robbie|
|    MEREDITH James|
+------------------+
only showing top 20 rows



3 Show FIFA Popular Name and 0 or 1 depending on Height>170

In [0]:
df = df.withColumn('Height_check',when(df['Height'] > 170,1).otherwise(0))
df.select('FIFA Popular Name','Height_check').show()

+------------------+------------+
| FIFA Popular Name|Height_check|
+------------------+------------+
|TAGLIAFICO Nicolas|           0|
|    PAVON Cristian|           0|
|    LANZINI Manuel|           0|
|    SALVIO Eduardo|           0|
|      MESSI Lionel|           0|
|  ANSALDI Cristian|           1|
|      BIGLIA Lucas|           1|
|       BANEGA Ever|           1|
| MASCHERANO Javier|           1|
|      DYBALA Paulo|           1|
|     AGUERO Sergio|           1|
|   HIGUAIN Gonzalo|           1|
|    DI MARIA Angel|           1|
|  LO CELSO Giovani|           1|
|  MEZA Maximiliano|           1|
|      ACUNA Marcos|           1|
|CABALLERO Wilfredo|           1|
|   MERCADO Gabriel|           1|
|  OTAMENDI Nicolas|           1|
|       ROJO Marcos|           1|
+------------------+------------+
only showing top 20 rows



4 name of shortest player

In [0]:
min_height=df.agg({'Height': 'min'}).collect()
df.select('FIFA Popular Name').filter(df['Height'] == min_height[0][0]).show()

+-----------------+
|FIFA Popular Name|
+-----------------+
| QUINTERO Alberto|
|   YAHIA ALSHEHRI|
|  SHAQIRI Xherdan|
+-----------------+



5 who is tallest of all. First we find the value of maximum height and then get the details of that player

In [0]:
max_height=df.agg({'Height': 'max'}).collect()
df.select('FIFA Popular Name').filter(df['Height'] == max_height[0][0]).show()

+-----------------+
|FIFA Popular Name|
+-----------------+
|    KALINIC Lovre|
+-----------------+



6 average height of the players in Argentina team.

In [0]:
df.agg({'Height': 'avg'}).show()

+-----------------+
|      avg(Height)|
+-----------------+
|182.4076086956522|
+-----------------+



Spark Level 2 SQL
Find the list of  oldest released movies.
How many movies are released each year?
How many number of movies are there for each rating?
How many users have rated each movie?
What is the total rating for each movie?
What is the average rating for each movie?

In [0]:
movies_df = spark.read.csv('/FileStore/tables/movies.csv',inferSchema=True,header=True)
ratings_df = spark.read.csv('/FileStore/tables/ratings.csv',inferSchema=True,header=True)

In [0]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [0]:
movies_df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [0]:
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows



In [0]:
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [0]:
movies_df.join(ratings_df, Seq("movieId", "movieId")).show

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-3401693147634536> in <module>
----> 1 movies_df.join(ratings_df, Seq("movieId", "movieId")).show

NameError: name 'Seq' is not defined

In [0]:
merged_df = movies_df.join(ratings_df,movies_df["movieId"] == ratings_df["movieId"],"inner").drop(ratings_df['movieId'])

In [0]:
merged_df.show()

+-------+--------------------+--------------------+------+------+----------+
|movieId|               title|              genres|userId|rating| timestamp|
+-------+--------------------+--------------------+------+------+----------+
|     31|Dangerous Minds (...|               Drama|     1|   2.5|1260759144|
|   1029|        Dumbo (1941)|Animation|Childre...|     1|   3.0|1260759179|
|   1061|     Sleepers (1996)|            Thriller|     1|   3.0|1260759182|
|   1129|Escape from New Y...|Action|Adventure|...|     1|   2.0|1260759185|
|   1172|Cinema Paradiso (...|               Drama|     1|   4.0|1260759205|
|   1263|Deer Hunter, The ...|           Drama|War|     1|   2.0|1260759151|
|   1287|      Ben-Hur (1959)|Action|Adventure|...|     1|   2.0|1260759187|
|   1293|       Gandhi (1982)|               Drama|     1|   2.0|1260759148|
|   1339|Dracula (Bram Sto...|Fantasy|Horror|Ro...|     1|   3.5|1260759125|
|   1343|    Cape Fear (1991)|            Thriller|     1|   2.0|1260759131|

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-3401693147634535> in <module>
----> 1 merged_df.drop(col('movieId'))

/databricks/spark/python/pyspark/sql/dataframe.py in drop(self, *cols)
   2732                 jdf = self._jdf.drop(col)
   2733             elif isinstance(col, Column):
-> 2734                 jdf = self._jdf.drop(col._jc)
   2735             else:
   2736                 raise TypeError("col should be a string or a Column")

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121                 # Hide where the exception came from that s

1 Find the list of oldest released movies

In [0]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format

In [0]:
movies_df.createOrReplaceTempView('Movies')
df=spark.sql('select *, CAST(substring(right(title,5),1,4) as integer) as year from Movies where title like "%(%)"')
df.createOrReplaceTempView('Movies')

In [0]:
df.show()

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
|      1|    Toy Story (1995)|Adventure|Animati...|1995|
|      2|      Jumanji (1995)|Adventure|Childre...|1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|1995|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|1995|
|      5|Father of the Bri...|              Comedy|1995|
|      6|         Heat (1995)|Action|Crime|Thri...|1995|
|      7|      Sabrina (1995)|      Comedy|Romance|1995|
|      8| Tom and Huck (1995)|  Adventure|Children|1995|
|      9| Sudden Death (1995)|              Action|1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|1995|
|     11|American Presiden...|Comedy|Drama|Romance|1995|
|     12|Dracula: Dead and...|       Comedy|Horror|1995|
|     13|        Balto (1995)|Adventure|Animati...|1995|
|     14|        Nixon (1995)|               Drama|1995|
|     15|Cutthroat Island ...|A

1

In [0]:
spark.sql('select * from movies order by year').show()

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
| 108548|Big Bang Theory, ...|              Comedy|null|
|  32898|Trip to the Moon,...|Action|Adventure|...|1902|
|   7065|Birth of a Nation...|           Drama|War|1915|
|   7243|Intolerance: Love...|               Drama|1916|
|  62383|20,000 Leagues Un...|Action|Adventure|...|1916|
|   8511|Immigrant, The (1...|              Comedy|1917|
|   3309|Dog's Life, A (1918)|              Comedy|1918|
|  72626|Billy Blazes, Esq...|      Comedy|Western|1919|
|   6987|Cabinet of Dr. Ca...|Crime|Fantasy|Horror|1920|
|  25737|Golem, The (Golem...|      Fantasy|Horror|1920|
|  59945|Mark of Zorro, Th...|Adventure|Romance...|1920|
|  83359|Play House, The (...|              Comedy|1921|
|  83096|Haunted House, Th...|              Comedy|1921|
|   3310|     Kid, The (1921)|        Comedy|Drama|1921|
|  83318|    Goat, The (1921)| 

2 How many movies are released each year

In [0]:
spark.sql('select year, count(year) as count_year from Movies group by year order by year').show()

+----+----------+
|year|count_year|
+----+----------+
|null|         0|
|1902|         1|
|1915|         1|
|1916|         2|
|1917|         1|
|1918|         1|
|1919|         1|
|1920|         3|
|1921|         5|
|1922|         6|
|1923|         3|
|1924|         5|
|1925|         8|
|1926|         6|
|1927|         8|
|1928|         5|
|1929|         7|
|1930|         5|
|1931|        14|
|1932|        14|
+----+----------+
only showing top 20 rows



3 How many number of movies are there for each rating

In [0]:
merged_df=movies_df.join(ratings_df,on='movieId',how='outer')

In [0]:
merged_df.show()

+-------+----------------+--------------------+------+------+----------+
|movieId|           title|              genres|userId|rating| timestamp|
+-------+----------------+--------------------+------+------+----------+
|      1|Toy Story (1995)|Adventure|Animati...|     7|   3.0| 851866703|
|      1|Toy Story (1995)|Adventure|Animati...|     9|   4.0| 938629179|
|      1|Toy Story (1995)|Adventure|Animati...|    13|   5.0|1331380058|
|      1|Toy Story (1995)|Adventure|Animati...|    15|   2.0| 997938310|
|      1|Toy Story (1995)|Adventure|Animati...|    19|   3.0| 855190091|
|      1|Toy Story (1995)|Adventure|Animati...|    20|   3.5|1238729767|
|      1|Toy Story (1995)|Adventure|Animati...|    23|   3.0|1148729853|
|      1|Toy Story (1995)|Adventure|Animati...|    26|   5.0|1360087980|
|      1|Toy Story (1995)|Adventure|Animati...|    30|   4.0| 944943070|
|      1|Toy Story (1995)|Adventure|Animati...|    37|   4.0| 981308121|
|      1|Toy Story (1995)|Adventure|Animati...|    

In [0]:
merged_df.createOrReplaceTempView('Movie_Data')
spark.sql('select rating, count(movieId) as count_movieID from Movie_Data group by rating order by rating').show()

+------+-------------+
|rating|count_movieID|
+------+-------------+
|  null|           59|
|   0.5|         1101|
|   1.0|         3326|
|   1.5|         1687|
|   2.0|         7271|
|   2.5|         4449|
|   3.0|        20064|
|   3.5|        10538|
|   4.0|        28750|
|   4.5|         7723|
|   5.0|        15095|
+------+-------------+



4 How many users have rated each movie

In [0]:
#4
spark.sql('select movieId, title, count(userId) as count_userID from Movie_Data group by movieId, title').show()

+-------+--------------------+------------+
|movieId|               title|count_userID|
+-------+--------------------+------------+
|    442|Demolition Man (1...|          95|
|    493|Menace II Society...|          23|
|    881|    First Kid (1996)|           4|
|    938|         Gigi (1958)|          11|
|   1165|Bloody Child, The...|           1|
|   1490|      B*A*P*S (1997)|           2|
|   1931|Mutiny on the Bou...|          10|
|   1943|Greatest Show on ...|           6|
|   2076|  Blue Velvet (1986)|          41|
|   2171|Next Stop Wonderl...|           4|
|   2201|Paradine Case, Th...|           2|
|   2370|Emerald Forest, T...|           3|
|   2440|Another Day in Pa...|           4|
|   2657|Rocky Horror Pict...|          63|
|   2824| On the Ropes (1999)|           2|
|   2907|    Superstar (1999)|           6|
|   2946|        Help! (1965)|          11|
|   3596|      Screwed (2000)|           1|
|   3883|Catfish in Black ...|           1|
|   4085|Beverly Hills Cop...|  

5 What is the total rating for each movie

In [0]:
spark.sql('select movieId, title, sum(rating) as sum_rating from Movie_Data group by movieId, title').show()

+-------+--------------------+----------+
|movieId|               title|sum_rating|
+-------+--------------------+----------+
|    442|Demolition Man (1...|     282.0|
|    493|Menace II Society...|      76.0|
|    881|    First Kid (1996)|      12.0|
|    938|         Gigi (1958)|      42.0|
|   1165|Bloody Child, The...|       1.5|
|   1490|      B*A*P*S (1997)|       1.5|
|   1931|Mutiny on the Bou...|      42.5|
|   1943|Greatest Show on ...|      20.0|
|   2076|  Blue Velvet (1986)|     165.5|
|   2171|Next Stop Wonderl...|      15.5|
|   2201|Paradine Case, Th...|       9.0|
|   2370|Emerald Forest, T...|      10.0|
|   2440|Another Day in Pa...|      16.0|
|   2657|Rocky Horror Pict...|     200.0|
|   2824| On the Ropes (1999)|       6.0|
|   2907|    Superstar (1999)|      18.0|
|   2946|        Help! (1965)|      39.0|
|   3596|      Screwed (2000)|       1.0|
|   3883|Catfish in Black ...|       0.5|
|   4085|Beverly Hills Cop...|     187.5|
+-------+--------------------+----

6 What is the average rating for each movie?

In [0]:
spark.sql('select movieId, title, avg(rating) as avg_rating from Movie_Data group by movieId, title').show()

+-------+--------------------+------------------+
|movieId|               title|        avg_rating|
+-------+--------------------+------------------+
|    442|Demolition Man (1...| 2.968421052631579|
|    493|Menace II Society...|3.3043478260869565|
|    881|    First Kid (1996)|               3.0|
|    938|         Gigi (1958)|3.8181818181818183|
|   1165|Bloody Child, The...|               1.5|
|   1490|      B*A*P*S (1997)|              0.75|
|   1931|Mutiny on the Bou...|              4.25|
|   1943|Greatest Show on ...|3.3333333333333335|
|   2076|  Blue Velvet (1986)| 4.036585365853658|
|   2171|Next Stop Wonderl...|             3.875|
|   2201|Paradine Case, Th...|               4.5|
|   2370|Emerald Forest, T...|3.3333333333333335|
|   2440|Another Day in Pa...|               4.0|
|   2657|Rocky Horror Pict...|3.1746031746031744|
|   2824| On the Ropes (1999)|               3.0|
|   2907|    Superstar (1999)|               3.0|
|   2946|        Help! (1965)|3.5454545454545454|
